
- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [183]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags

In [184]:
# load one file first to see what type of data you're dealing with and what attributes it has

directory = '../data/'

with open(f'{directory}AK_Juneau_0.json') as f:
    jsonfile = json.load(f)

testcity = pd.json_normalize(jsonfile['data']['results']).head(3)

jsoncolumns = testcity.columns

testcity

,last_update_date,tags,permalink,status,list_date,open_houses,branding,list_price,property_id,photos,...,location.address.state_code,location.address.line,location.street_view_url,location.county.fips_code,location.county.name,primary_photo,source,products,location.address.coordinate,other_listings
0,2023-09-19T20:52:50Z,"[carport, community_outdoor_space, cul_de_sac,...",9453-Herbert-Pl_Juneau_AK_99801_M90744-30767,sold,2023-06-29T21:16:25.000000Z,None,"[{'name': 'EXP Realty LLC - Southeast Alaska',...",554950.0,9074430767,"[{'tags': [{'label': 'house_view', 'probabilit...",...,AK,9453 Herbert Pl,https://maps.googleapis.com/maps/api/streetvie...,None,Juneau,NaN,NaN,NaN,NaN,NaN
1,None,None,8477-Thunder-Mountain-Rd_Juneau_AK_99801_M9424...,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,9424983842,None,...,AK,8477 Thunder Mountain Rd,https://maps.googleapis.com/maps/api/streetvie...,None,Juneau,NaN,NaN,NaN,NaN,NaN
2,None,None,4515-Glacier-Hwy_Juneau_AK_99801_M94790-68516,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,9479068516,None,...,AK,4515 Glacier Hwy,https://maps.googleapis.com/maps/api/streetvie...,None,Juneau,NaN,NaN,NaN,NaN,NaN


In [185]:
import warnings

df = pd.DataFrame(columns= jsoncolumns)

json_files = [file for file in os.listdir(directory) if file.endswith('.json')]

for file in json_files:
    file_path = os.path.join(directory, file)
    json_subfile = pd.read_json(file_path)
    json_subdf = pd.json_normalize(json_subfile['data']['results'])
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        df = pd.concat([df, json_subdf], ignore_index=True)
    
df.shape

(8191, 67)

In [186]:
[col for col in df.columns if col not in jsoncolumns]

['community.advertisers', 'community.description.name', 'location.county']

At this point, ensure that you have all sales in a dataframe.
- Is each cell one value, or do some cells have lists?
- Maybe the "tags" will help create some features.
- What are the data types of each column?
- Some sales may not actually include the sale price.  These rows should be dropped.
- Some sales don't include the property type.
- There are a lot of None values.  Should these be dropped or replaced with something?

In [187]:
df.shape

(8191, 67)

In [188]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8191 entries, 0 to 8190
Data columns (total 67 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   last_update_date                       8125 non-null   object 
 1   tags                                   7638 non-null   object 
 2   permalink                              8159 non-null   object 
 3   status                                 8159 non-null   object 
 4   list_date                              7752 non-null   object 
 5   open_houses                            0 non-null      object 
 6   branding                               8159 non-null   object 
 7   list_price                             7721 non-null   object 
 8   property_id                            8159 non-null   object 
 9   photos                                 7403 non-null   object 
 10  community                              0 non-null      object 
 11  virt

In [189]:
df.describe()

,price_reduced_amount,description.year_built,description.baths_3qtr,description.baths_full,description.baths_half,description.lot_sqft,description.sqft,description.baths,description.garage,description.stories,description.beds,location.address.coordinate.lon,location.address.coordinate.lat,location.county
count,2.484000e+03,7316.000000,566.000000,7311.000000,2281.000000,6.991000e+03,7323.000000,7980.000000,4448.000000,6260.000000,7504.000000,7909.000000,7909.000000,0.0
mean,2.442704e+04,1968.916074,1.247350,1.887430,1.105655,2.510949e+05,1933.848559,2.131203,1.926709,1.567732,3.208289,-92.206522,39.009689,NaN
std,7.162396e+04,35.096914,0.463482,0.862214,0.413340,5.823820e+06,1339.039206,1.175940,0.878766,0.730969,1.282732,15.888886,4.374553,NaN
min,1.000000e+02,1828.000000,1.000000,1.000000,1.000000,0.000000e+00,120.000000,0.000000,1.000000,1.000000,0.000000,-157.810583,21.277707,NaN
25%,6.000000e+03,1950.000000,1.000000,1.000000,1.000000,4.953000e+03,1258.000000,1.000000,1.000000,1.000000,3.000000,-104.971611,35.688084,NaN
50%,1.010000e+04,1975.000000,1.000000,2.000000,1.000000,7.841000e+03,1635.000000,2.000000,2.000000,1.000000,3.000000,-89.333131,39.698210,NaN
75%,2.000000e+04,1997.000000,1.000000,2.000000,1.000000,1.263200e+04,2264.000000,3.000000,2.000000,2.000000,4.000000,-78.617690,41.832266,NaN
max,2.015999e+06,2024.000000,3.000000,8.000000,5.000000,1.676624e+08,32106.000000,9.000000,11.000000,10.000000,12.000000,-71.006343,58.396178,NaN


In [190]:
# load and concatenate data here
# drop or replace values as necessary

print(df.iloc[0,0:10])
print(df.iloc[0,10:20])
print(df.iloc[0,20:30])



last_update_date                                 2024-01-17T00:01:55Z
tags                [central_air, dishwasher, fireplace, forced_ai...
permalink             1869-Pebble-Ct_Harrisburg_PA_17110_M33009-59879
status                                                           sold
list_date                                 2023-11-27T05:15:07.000000Z
open_houses                                                      None
branding            [{'name': 'Howard Hanna - Colonial Park', 'pho...
list_price                                                     334900
property_id                                                3300959879
photos              [{'tags': [{'label': 'garage', 'probability': ...
Name: 0, dtype: object
community                                                            None
virtual_tours                                                        None
listing_id                                                     2961973062
price_reduced_amount                                   

In [191]:
print(df.iloc[0,30:40])
print(df.iloc[0,40:50])
print(df.iloc[0,50:60])
print(df.iloc[0,60:70])

description.sub_type                              None
description.baths_1qtr                            None
description.garage                                 2.0
description.stories                                1.0
description.beds                                   3.0
description.type                         single_family
lead_attributes.show_contact_an_agent             True
flags.is_new_construction                         None
flags.is_for_rent                                 None
flags.is_subdivision                              None
Name: 0, dtype: object
flags.is_contingent                                                          None
flags.is_price_reduced                                                       None
flags.is_pending                                                             None
flags.is_foreclosure                                                         None
flags.is_plan                                                                None
flags.is_coming_s

In [192]:
print(df['tags'][0])

['central_air', 'dishwasher', 'fireplace', 'forced_air', 'hardwood_floors', 'washer_dryer', 'basement', 'single_story', 'garage_1_or_more', 'garage_2_or_more', 'big_yard', 'vaulted_ceiling', 'open_floor_plan', 'floor_plan', 'corner_lot', 'big_lot', 'ensuite']


In [193]:
print(df['branding'][0])

[{'name': 'Howard Hanna - Colonial Park', 'photo': None, 'type': 'Office'}]


In [194]:
print(df['photos'][0]) 

[{'tags': [{'label': 'garage', 'probability': 0.99981635808945}, {'label': 'house_view', 'probability': 0.99869686365128}, {'label': 'road_view', 'probability': 0.99956554174423}, {'label': 'yard', 'probability': 0.9751615524292}, {'label': 'house_view', 'probability': 0.99909782409668}, {'label': 'house_view', 'probability': 0.99994349479675}], 'href': 'https://ap.rdcpix.com/8c0b391ae4f9bbca36c41d559ef75598l-m2349710986s-w1024_h768.jpg'}]


In [195]:
# [value for value in df['price_reduced_amount'] if value is not np.NaN]


In [196]:

drop_cols = ['last_update_date', 'permalink', 'list_date', 'open_houses', 'branding', 
             'property_id', 'photos', 'community', 'virtual_tours', 'listing_id', 'matterport',
             'primary_photo.href', 'source.plan_id', 'source.agents', 'source.spec_id', 
             'source.type', 'description.name','lead_attributes.show_contact_an_agent',
               'price_reduced_amount', 'flags.is_for_rent', 'flags.is_contingent',
               'flags.is_pending', 'flags.is_plan', 'flags.is_coming_soon', 'flags.is_new_listing',
               'products.brand_name', 'other_listings.rdc', 'location.address.postal_code', 'location.address.line',
               'location.street_view_url', 'location.county.fips_code', 'primary_photo', 'source', 'products',
               'location.address.coordinate', 'other_listings', 'community.advertisers', 'community.description.name',
               'location.county', 'status', 'location.address.state', 'flags.is_new_construction', 
               'flags.is_subdivision', 'flags.is_price_reduced', 'flags.is_foreclosure']

            

df = df.drop(columns= drop_cols)

In [197]:
df.shape

(8191, 22)

In [206]:
df.iloc[0:5, :10]

,tags,list_price,description.year_built,description.baths_3qtr,description.sold_date,description.sold_price,description.baths_full,description.baths_half,description.lot_sqft,description.sqft
0,"[central_air, dishwasher, fireplace, forced_ai...",334900,2001.0,NaN,2024-01-16,345000,2.0,NaN,22651.0,1539.0
1,"[disability_features, basement, garage_1_or_mo...",185000,1900.0,NaN,2024-01-16,196000,NaN,NaN,2614.0,2429.0
2,"[carport, central_air, dishwasher, fireplace, ...",197000,1971.0,NaN,2024-01-12,205000,2.0,NaN,13504.0,1120.0
3,"[central_air, dining_room, disability_features...",295000,2013.0,NaN,2024-01-12,295000,2.0,1.0,2688.0,2400.0
4,"[dining_room, hardwood_floors, basement]",105000,1900.0,NaN,2024-01-12,106000,NaN,NaN,871.0,1478.0


In [199]:
df.iloc[0:5,10:]

,description.baths,description.sub_type,description.baths_1qtr,description.garage,description.stories,description.beds,description.type,location.address.coordinate.lon,location.address.coordinate.lat,location.address.city,location.address.state_code,location.county.name
0,2.0,None,None,2.0,1.0,3.0,single_family,-76.865096,40.316612,Harrisburg,PA,Dauphin
1,NaN,None,None,3.0,NaN,3.0,multi_family,-76.900123,40.296987,Harrisburg,PA,Dauphin
2,2.0,None,None,NaN,1.0,3.0,single_family,-76.757031,40.257449,Harrisburg,PA,Dauphin
3,3.0,townhouse,None,1.0,2.0,3.0,townhomes,-76.821034,40.350224,Harrisburg,PA,Dauphin
4,NaN,None,None,NaN,NaN,3.0,multi_family,-76.869911,40.259374,Harrisburg,PA,Dauphin


In [200]:
# df['flags.is_price_reduced'].value_counts()/df.shape[0]

In [201]:
# df['flags.is_foreclosure'].value_counts()/df.shape[0]

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.
- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE such as using central tendency?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.

In [202]:
# OHE categorical variables here
# tags will have to be done manually

If using numerical values to replace cities or states, make sure that the data is split so that we don't leak data into the training selection.
- If you think about a real scenario, do we have any knowledge of new listings that would come in once modeling is complete?
- Drop columns that aren't needed.
- Don't keep the list price because it will be too close to the sale price.

In [203]:
# perform train test split here
# do something with state and city

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [204]:
# perform EDA here

Now is a great time to scale the data and save it once it's preprocessed.